In [2]:
import numpy as np
# import tensorflow as tf
import librosa
import note_seq
import bokeh
from note_seq import NoteSequence
from typing import Iterable, List, Tuple


import matplotlib.pyplot as plt

In [83]:
# np.random.uniform(1,128)

59.64386343211563

In [100]:
def generate_midi(duration: int, time_step: int, max_num_pitches: int) -> List:
    ns = NoteSequence()
    num_parts = duration / time_step

    for i in np.arange(num_parts):
        # frame_time = steps[i + 1] - steps[i]
        # time = steps[i]

        for _ in np.arange(np.random.choice(max_num_pitches)):
            start_time = time_step * i
            end_time = start_time + np.random.uniform(0, time_step)
            end_time = end_time if end_time <= 100 else 100
            ns.notes.append(
                NoteSequence.Note(
                    pitch=np.random.choice(128),
                    velocity=np.random.choice(np.arange(1,128)),
                    start_time=start_time,
                    end_time=end_time,
                )
            )

    return ns


gen_midi = generate_midi(duration=100, time_step=15, max_num_pitches=10)
note_seq.plot_sequence(gen_midi)

Loading BokehJS ...